In [1]:
# EU 
# For One Ye

Vorgehnsweise: 
Visualisierung in Pandas Darstellen 
Probe Dashboard mit plotly Dash erstellen 
Dashboar in Jupyter Book ein binden 
-> weiteren Grafiken umsetzen 

Ziele: 
Auswertung auf Basis EU 

Zeitlicher Verlauf gesamt
Abfragen:
    - Summe burned area pro Monat 
    - Fläche verbrannter Wälder insgesamt 
    - Liste besonders betroffener Länder 

Visualisierungen: 
    - Balken-Diagram: Summe burned area pro Monat 
    - Treemap: Verbrannte Fläche pro Vegetationstyp
    - Weltkarte mit betroffener Länder (gefüllt): Farbe Burned Area
    - DE-Karte: Bubles Größe (Burned area)


In [2]:
from pymongo import MongoClient
import pandas as pd
import plotly.express as px

In [3]:
!"C:\Program Files\MongoDB\Server\6.0\bin\mongod.exe" --dbpath "C:\mongodb_data"

{"t":{"$date":"2022-09-10T12:17:00.577+02:00"},"s":"I",  "c":"NETWORK",  "id":4915701, "ctx":"-","msg":"Initialized wire specification","attr":{"spec":{"incomingExternalClient":{"minWireVersion":0,"maxWireVersion":17},"incomingInternalClient":{"minWireVersion":0,"maxWireVersion":17},"outgoing":{"minWireVersion":6,"maxWireVersion":17},"isInternalClient":true}}}
{"t":{"$date":"2022-09-10T12:17:02.110+02:00"},"s":"I",  "c":"CONTROL",  "id":23285,   "ctx":"-","msg":"Automatically disabling TLS 1.0, to force-enable TLS 1.0 specify --sslDisabledProtocols 'none'"}
{"t":{"$date":"2022-09-10T12:17:02.112+02:00"},"s":"I",  "c":"NETWORK",  "id":4648602, "ctx":"main","msg":"Implicit TCP FastOpen in use."}
{"t":{"$date":"2022-09-10T12:17:02.115+02:00"},"s":"I",  "c":"REPL",     "id":5123008, "ctx":"main","msg":"Successfully registered PrimaryOnlyService","attr":{"service":"TenantMigrationDonorService","namespace":"config.tenantMigrationDonors"}}
{"t":{"$date":"2022-09-10T12:17:02.115+02:00"},"s":"I

In [4]:
client = MongoClient()
client = MongoClient('mongodb://localhost:27017/')
mydb = client["esa_fire"]
mycol = mydb["esa_fire_2001"]
print(mydb.name)

esa_fire


### Query 1: Flächen in Deutschland mit einer verbrannten Fläche größer als 1 km2

In [5]:
# Querys Mongo DB
query2={
     '$and': [
            { 'country' : 'DE' },
            { 'burned_area'    : { '$gt': 1000000 }}
          ]
}
points =mycol.find(query2)
print("Anzahl von Städten mit Bevölkerungsdichte größer als 2000 in BaWü: ",mycol.count_documents(query2))

Anzahl von Städten mit Bevölkerungsdichte größer als 2000 in BaWü:  43


In [6]:
import folium

In [7]:
m = folium.Map(location=[48.74388,9.10014],zoom_start=5)

In [8]:
for citie in points:
    folium.Marker( location=[ citie['lat'], citie['lon'] ], fill_color='#43d9de', radius=8,tooltip=citie['date']).add_to(m)

In [9]:
#m

### Query 2: Gruppierung der Flächen nach Datum

In [10]:
list_date = list ()
agg_result= mycol.aggregate(
    [{
    "$group" : 
        {"_id" : "$date", 
         "burned_area" : {"$sum" : 1}
         }
    }])
for i in agg_result:
    list_date.append(i)
list_date

[{'_id': '2016-12-01', 'burned_area': 8845},
 {'_id': '2012-10-01', 'burned_area': 15972},
 {'_id': '2005-05-01', 'burned_area': 13805},
 {'_id': '2015-04-01', 'burned_area': 11639},
 {'_id': '2020-11-01', 'burned_area': 10493},
 {'_id': '2003-06-01', 'burned_area': 12926},
 {'_id': '2012-12-01', 'burned_area': 9603},
 {'_id': '2008-06-01', 'burned_area': 12639},
 {'_id': '2014-01-01', 'burned_area': 9948},
 {'_id': '2001-02-01', 'burned_area': 8887},
 {'_id': '2013-01-01', 'burned_area': 10389},
 {'_id': '2014-12-01', 'burned_area': 8718},
 {'_id': '2005-01-01', 'burned_area': 10996},
 {'_id': '2009-09-01', 'burned_area': 18170},
 {'_id': '2009-11-01', 'burned_area': 11440},
 {'_id': '2003-08-01', 'burned_area': 21014},
 {'_id': '2013-06-01', 'burned_area': 11081},
 {'_id': '2020-06-01', 'burned_area': 11732},
 {'_id': '2011-02-01', 'burned_area': 9746},
 {'_id': '2015-01-01', 'burned_area': 9939},
 {'_id': '2002-03-01', 'burned_area': 7938},
 {'_id': '2017-06-01', 'burned_area': 1102

In [11]:
import pandas as pd
df_date = pd.DataFrame(list_date)

In [12]:
import plotly.express as px

fig = px.bar(df_date, x="_id", y="burned_area", title='Life expectancy in Canada')
fig.show()

In [13]:
list_date = list ()
agg_result= mycol.aggregate(
    [{
    "$group" : 
        {"_id" : "$country", 
         "burned_area" : {"$sum" : 1}
         }
    }])
for i in agg_result:
    list_date.append(i)
list_date

[{'_id': 'MY', 'burned_area': 2572},
 {'_id': 'GY', 'burned_area': 2968},
 {'_id': 'FJ', 'burned_area': 865},
 {'_id': 'JP', 'burned_area': 402},
 {'_id': 'TD', 'burned_area': 42525},
 {'_id': 'VU', 'burned_area': 229},
 {'_id': 'LB', 'burned_area': 117},
 {'_id': 'ET', 'burned_area': 39221},
 {'_id': 'ML', 'burned_area': 48554},
 {'_id': 'PH', 'burned_area': 4896},
 {'_id': 'GN', 'burned_area': 31912},
 {'_id': 'SO', 'burned_area': 1050},
 {'_id': 'PR', 'burned_area': 101},
 {'_id': 'TT', 'burned_area': 127},
 {'_id': 'BR', 'burned_area': 297973},
 {'_id': 'SS', 'burned_area': 77883},
 {'_id': 'LA', 'burned_area': 6541},
 {'_id': 'KG', 'burned_area': 2207},
 {'_id': 'RS', 'burned_area': 2960},
 {'_id': 'NE', 'burned_area': 1702},
 {'_id': 'EG', 'burned_area': 664},
 {'_id': 'MK', 'burned_area': 852},
 {'_id': 'TW', 'burned_area': 122},
 {'_id': 'NL', 'burned_area': 32},
 {'_id': 'UA', 'burned_area': 55906},
 {'_id': 'BZ', 'burned_area': 924},
 {'_id': 'AT', 'burned_area': 103},
 {'_id

In [14]:
df_date = pd.DataFrame(list_date).sort_values(by=['burned_area'], ascending= False)
df_date

,_id,burned_area
79,RU,379365
14,BR,297973
29,AU,231628
86,CD,144610
57,ZM,135869
...,...,...
73,CW,1
90,DM,1
121,MT,1
54,VI,1


In [15]:
fig = px.bar(df_date, x="_id", y="burned_area", title='Life expectancy in Canada')
fig.show()

## Verbrannte Fläche pro Vegetationstyp

In [16]:
# Summe der Verbranten Fläche ->  pro Vegetatoionstyp
# Waterfall Diagramm

In [17]:

list_veg = list ()
agg_result= mycol.aggregate([{ "$group":
  { "_id" : None,  
    "burned_area"  : { "$sum": "$burned_area" }, 
    "no_data_burned_area"  : { "$sum":"$no_data_burned_area" },
    "tree_cover_broadleaved_deciduous_burned_area"  : { "$sum":"$tree_cover_broadleaved_deciduous_burned_area" },
    "tree_cover_broadleaved_deciduous_burned_area"  : { "$sum":"$tree_cover_broadleaved_deciduous_burned_area" },
    "tree_cover_needleleaved_evergreen_burned_area"  : { "$sum":"$tree_cover_needleleaved_evergreen_burned_area" },
    "tree_cover_needleleaved_deciduous_burned_area"  : { "$sum":"$tree_cover_needleleaved_deciduous_burned_area" },
    "tree_cover_mixed_leave_burned_area"  : { "$sum":"$tree_cover_mixed_leave_burned_area"},
    "cropland_burned_area"  : { "$sum":"$cropland_burned_area" },
    "mosaic_tree_grass_burned_area"  : { "$sum":"$mosaic_tree_grass_burned_area" },
    "flooded_area_burned_area"  : { "$sum":"$flooded_area_burned_area" },
    "other_vegetation_burned_area"  : { "$sum":"$other_vegetation_burned_area" },}
}])
for i in agg_result:
    list_veg.append(i)
list_veg

[{'_id': None,
  'burned_area': 89940704892432.0,
  'no_data_burned_area': 7004942864842.0,
  'tree_cover_broadleaved_deciduous_burned_area': 35183926581228.0,
  'tree_cover_needleleaved_evergreen_burned_area': 807798707721.0,
  'tree_cover_needleleaved_deciduous_burned_area': 988045537799.0,
  'tree_cover_mixed_leave_burned_area': 151480670524.0,
  'cropland_burned_area': 5687008018231.0,
  'mosaic_tree_grass_burned_area': 4188674581006.0,
  'flooded_area_burned_area': 2544568553482.0,
  'other_vegetation_burned_area': 31979288438507.0}]

In [18]:
df_veg = pd.DataFrame(list_veg)

In [19]:
df_veg= df_veg.drop(columns='_id')

In [20]:
fig = px.bar(df_veg, title='Life expectancy in Canada')
fig.show()

In [21]:
import plotly.graph_objects as go

fig1  = go.Figure()
hrz = ["burned_area", "no_data_burned_area","tree_cover_broadleaved_deciduous_burned_area",
       "tree_cover_needleleaved_evergreen_burned_area","tree_cover_needleleaved_deciduous_burned_area", "cropland_burned_area", "mosaic_tree_grass_burned_area", "flooded_area_burned_area", "other_vegetation_burned_area"
       "Final"]
vrt = [ df_veg["burned_area"][0], 
       -df_veg["no_data_burned_area"][0],
       -df_veg["tree_cover_broadleaved_deciduous_burned_area"][0],
       -df_veg["tree_cover_needleleaved_evergreen_burned_area"][0],
       -df_veg["tree_cover_needleleaved_deciduous_burned_area"][0],
       -df_veg["cropland_burned_area"][0],
       -df_veg["mosaic_tree_grass_burned_area"][0],
       -df_veg["flooded_area_burned_area"][0],
       -df_veg["other_vegetation_burned_area"][0],
       None]
fig1.add_trace(go.Waterfall(                 
                  x = hrz, y = vrt,
                  base = 0,
                  measure = [ "absolute","relative",    
                              "relative","relative",
                              "relative","relative" ]                        
                ))                                 
fig1.show()



In [22]:
df_veg= df_veg.drop(columns='burned_area')

In [23]:
df_veg_trans = df_veg.transpose()

In [24]:
df_veg_trans.reset_index(inplace = True)
df_veg_trans

,index,0
0,no_data_burned_area,7.004943e+12
1,tree_cover_broadleaved_deciduous_burned_area,3.518393e+13
2,tree_cover_needleleaved_evergreen_burned_area,8.077987e+11
3,tree_cover_needleleaved_deciduous_burned_area,9.880455e+11
4,tree_cover_mixed_leave_burned_area,1.514807e+11
5,cropland_burned_area,5.687008e+12
6,mosaic_tree_grass_burned_area,4.188675e+12
7,flooded_area_burned_area,2.544569e+12
8,other_vegetation_burned_area,3.197929e+13


In [25]:
fig = px.treemap(df_veg_trans, 
    path = [px.Constant('All'), 'index'], 
    color = 0, 
    color_continuous_scale='Emrld',
    values = 0,
    width = 800,
    height = 600)
fig.show()

In [26]:
# Abfrage EU Länder
# Umwandlung in Dataframe

## Dashboard Europa

In [27]:
df_countries = pd.read_csv('C:\dev\eda_esa_fire\doc\country_codes.csv', on_bad_lines='skip', delimiter=',')

In [28]:
europe = df_countries[df_countries['region'] == 'Europe']['alpha-2'].to_list()

In [29]:
list_europe = list(mycol.find({"country":{"$in":europe}}))
df_eu = pd.DataFrame(list_europe)

In [30]:
df_eu

,_id,lon,lat,country,date,burned_area,standard_error,fraction_of_burnable_area,fraction_of_observed_area,number_of_patches,no_data_burned_area,tree_cover_broadleaved_evergreen_burned_area,tree_cover_broadleaved_deciduous_burned_area,tree_cover_needleleaved_evergreen_burned_area,tree_cover_needleleaved_deciduous_burned_area,tree_cover_mixed_leave_burned_area,cropland_burned_area,mosaic_tree_grass_burned_area,other_vegetation_burned_area,flooded_area_burned_area
0,630630b328dd00e55a25462f,-3.625,56.125,GB,2001-01-01,804970.0,545465.0,0.829803,0.959910,3.0,751305.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53664.0,0.0
1,630630b328dd00e55a254630,-3.625,55.875,GB,2001-01-01,2039257.0,707836.0,0.938498,0.977980,1.0,1502610.0,0.0,0.0,0.0,0.0,0.0,375652.0,0.0,160994.0,0.0
2,630630b328dd00e55a254631,22.875,53.125,PL,2001-01-01,3005221.0,838988.0,0.996411,0.981294,1.0,0.0,0.0,590311.0,0.0,0.0,0.0,0.0,0.0,53664.0,2361245.0
3,630630b328dd00e55a254632,24.625,51.875,UA,2001-01-01,1609940.0,720975.0,0.974907,0.963874,3.0,965964.0,0.0,429317.0,0.0,0.0,0.0,0.0,0.0,214658.0,0.0
4,630630b328dd00e55a254633,26.125,51.875,UA,2001-01-01,21948848.0,929124.0,0.991455,0.826491,2.0,1287952.0,0.0,268323.0,0.0,0.0,0.0,697640.0,0.0,1985592.0,17709340.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490553,630634c328dd00e55a5607e6,-7.875,41.375,PT,2020-12-01,3166215.0,979340.0,0.985474,1.000000,3.0,0.0,0.0,0.0,2146586.0,0.0,214658.0,0.0,1609940.0,0.0,0.0
490554,630634c328dd00e55a5607ed,-4.375,40.375,ES,2020-12-01,429317.0,552708.0,0.972653,1.000000,2.0,0.0,0.0,0.0,107329.0,0.0,0.0,53664.0,536646.0,0.0,0.0
490555,630634c328dd00e55a5607f1,-8.125,40.125,PT,2020-12-01,2790562.0,557590.0,0.985990,1.000000,27.0,0.0,0.0,0.0,2361245.0,0.0,321988.0,0.0,214658.0,0.0,0.0
490556,630634c328dd00e55a5607f4,8.625,39.875,IT,2020-12-01,3166215.0,515265.0,0.802660,1.000000,8.0,214658.0,0.0,0.0,0.0,0.0,0.0,2844227.0,0.0,107329.0,0.0


In [31]:
df_eu['year'] = df_eu['date'].str.slice(start=0, stop=4)

In [32]:
df_test = df_eu[(df_eu['year'] == '2016') & (df_eu['country'] == 'IT')]

## Test Graphics

In [33]:
df_test2 = df_test.groupby(by= 'date').sum().reset_index()
df_test2

,date,lon,lat,burned_area,standard_error,fraction_of_burnable_area,fraction_of_observed_area,number_of_patches,no_data_burned_area,tree_cover_broadleaved_evergreen_burned_area,tree_cover_broadleaved_deciduous_burned_area,tree_cover_needleleaved_evergreen_burned_area,tree_cover_needleleaved_deciduous_burned_area,tree_cover_mixed_leave_burned_area,cropland_burned_area,mosaic_tree_grass_burned_area,other_vegetation_burned_area,flooded_area_burned_area
0,2016-01-01,11.375,45.875,375652.0,529075.0,0.983945,0.990575,3.0,0.0,0.0,0.0,375652.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2016-05-01,29.500,74.500,1019628.0,1458540.0,1.926052,1.986665,5.0,912298.0,0.0,0.0,0.0,0.0,0.0,107328.0,0.0,0.0,0.0
2,2016-06-01,428.875,1127.875,78672391.0,18440286.0,21.852823,28.662573,70.0,40248490.0,0.0,536645.0,0.0,0.0,0.0,18836292.0,18997286.0,9552306.0,0.0
3,2016-07-01,1585.500,4342.000,532514454.0,83274285.0,83.389457,108.408936,521.0,390786071.0,0.0,5151803.0,4454164.0,0.0,0.0,36116290.0,29837538.0,79316358.0,1770933.0
4,2016-08-01,1662.125,4618.125,548667527.0,83971056.0,89.256920,113.412398,763.0,407261128.0,0.0,18192312.0,2092919.0,0.0,0.0,46741890.0,38531206.0,55113589.0,0.0
5,2016-09-01,992.000,2822.500,233333952.0,47555757.0,59.698495,69.181151,580.0,200974157.0,0.0,7942362.0,590310.0,0.0,0.0,8586333.0,7083726.0,11698886.0,0.0
6,2016-10-01,474.125,1286.125,343239206.0,28420756.0,28.558763,32.737365,201.0,323275949.0,0.0,0.0,0.0,0.0,0.0,7566712.0,7405720.0,8640008.0,0.0
7,2016-11-01,70.625,200.625,2092920.0,2967555.0,3.991025,4.940316,9.0,1448944.0,0.0,160994.0,0.0,0.0,0.0,0.0,0.0,482982.0,0.0


In [34]:

fig = px.area(df_test2, x="date", y="burned_area")
fig.update_traces(line_color='#FD7207')
fig.update_layout(title = 'Burend_Area', plot_bgcolor= "rgb(250, 250, 250)",
paper_bgcolor="rgb(250, 250, 250)",)
fig.show()

In [35]:
df_veg_group = df_test.groupby(by= 'year').sum().reset_index()
df_veg_group= df_veg_group.drop(columns= ['burned_area', 'year', 'lon', 'lat', 'standard_error', 'fraction_of_burnable_area', 'fraction_of_observed_area', 'number_of_patches'] )
df_veg_trans = df_veg_group.transpose().reset_index()
df_veg_trans = df_veg_trans[(df_veg_trans[0] != 0)]
fig = px.treemap(df_veg_trans, 
    path = [px.Constant('All'), 'index'], 
    color = 0, 
    color_continuous_scale='Emrld',
    values = 0,
    width = 800,
    height = 600)
fig.show()

In [36]:
df_labels = df_countries[df_countries['region'] == 'Europe'][['name', 'alpha-2']]
df_labels.rename(columns={'name': 'label', 'alpha-2': 'value'}, inplace = True)
list_options = df_labels.to_dict('records')

In [37]:
unique_dates  = df_eu['year'].unique()
option_dates =  list()
for data in unique_dates: 
    date_dict = {'label': data, 'value': data}
    option_dates.append(date_dict)


In [68]:

import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output# Load Data
from plotly.subplots import make_subplots

app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("Wild Fires Europe", style = {
                    'color': '#3C5C79',
                    'backgroundColor': "#fafafa",
                    'font-family': '"Open Sans", verdana, arial, sans-serif'
                }),
    html.Label([
        "Country",
    dcc.Dropdown(id="selected_country",
                 options= list_options,
                 multi=False,
                 style={'width': "40%"}
                 )],style = {
                    'color': '#3C5C79',
                    'backgroundColor': "#fafafa",
                    'font-family': '"Open Sans", verdana, arial, sans-serif'
                }),
    html.Label([ "Years",
    dcc.Dropdown(id= "selected_year",
                 options= option_dates,
                 multi=False,
                 style={'width': "40%"}
                 ), ], style = {
                    'color': '#3C5C79',
                    'backgroundColor': "#fafafa",
                    'font-family': '"Open Sans", verdana, arial, sans-serif'
                }), 
    html.Div([ 
            html.Div([
                dcc.Graph(id = 'fig_1'),
            ], className='three columns'),
            html.Div([
                dcc.Graph(id = 'fig_2' ),
            ], className='three columns'),
            html.Div([
                dcc.Graph(id = 'fig_3' ),
            ], className='three columns'),
        ], className = 'row'),
    ])


@app.callback(
    [Output('fig_1' , 'figure'),
     Output('fig_2' , 'figure'), 
     Output('fig_3' , 'figure')],
    Input("selected_country", "value"), 
    Input("selected_year", "value")
)


def update_figure(selected_country, selected_year):

    if not selected_country:
        selected_country = 'IT'
        selected_year = '2016'
    dff = df_eu.copy()
    dff = dff[dff["country"] == selected_country]
    dff = dff[dff["year"] == selected_year]
    


    fig = go.Figure(data=go.Scattergeo(
         lon=dff["lon"],
         lat= dff["lat"], 
         mode = 'markers',
         marker = dict(
            size = 8,
            opacity = 0.8,
            reversescale = True,
            autocolorscale = False,
            symbol = 'square',
            line = dict(
                width=0.5,
                color='rgba(102, 102, 102)'
            ),
            colorscale = 'Hot',
            cmin = 0,
            color = dff['burned_area'],
            cmax = dff['burned_area'].max(),
            colorbar_title="Burend Area"
        ))
    )
    fig.update_layout(
        title = 'Most trafficked US airports<br>(Hover for airport names)',
        geo = dict(
            scope='europe',
            showland = True,
            landcolor = "rgb(250, 250, 250)",
            subunitcolor = "rgb(217, 217, 217)",
            countrycolor = "rgb(217, 217, 217)",
            countrywidth = 0.5,
            subunitwidth = 0.5
        ),)
    fig_1 = fig

    df_time = dff.groupby(by= 'date').sum().reset_index()
    
    fig_2= px.area(df_time, x="date", y="burned_area")
    fig_2.update_traces(line_color='#FD7207')
    fig_2.update_layout(title = 'Burend_Area', plot_bgcolor= "rgb(250, 250, 250)", paper_bgcolor="rgb(250, 250, 250)",)
    
    df_veg_group = dff.groupby(by= 'year').sum().reset_index()
    df_veg_group= df_veg_group.drop(columns= ['burned_area', 'year', 'lon', 'lat', 'standard_error', 'fraction_of_burnable_area', 'fraction_of_observed_area', 'number_of_patches'] )
    df_veg_t = df_veg_group.transpose().reset_index()
    df_veg_t = df_veg_t[(df_veg_t[0] != 0)]
    fig_3 = px.treemap(df_veg_t, 
        path = [px.Constant('All'), 'index'], 
        color = 0, 
        color_continuous_scale='Emrld',
        values = 0)
    return [fig_1, fig_2, fig_3]
app.run_server(mode='inline')
